# Recall live tonnage list

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/Recall%20live%20Tonnage%20List.ipynb). 

## Setup
Install the Signal Ocean SDK:

In [ ]:
!pip install signal-ocean

Set your subscription key, acquired here: [https://apis.signalocean.com/profile](https://apis.signalocean.com/profile)

In [27]:
signal_ocean_api_key = ""  # replace with your subscription key

First, we need to create an instance of the `TonnageListAPI`:

In [28]:
from signal_ocean import Connection
from signal_ocean.tonnage_list import TonnageListAPI

connection = Connection(signal_ocean_api_key)
api = TonnageListAPI(connection)

## Retrieving a live tonnage list
Retrieving a live tonnage list is almost exactly the same as getting a historical one except, instead of using the `get_historical_tonnage_list` method, you use the `get_tonnage_list` method and you don't pass a `DateRange` as an argument. The `get_tonnage_list` method returns a single `TonnageList` that contains live vessel data.

Because of this similarity, we can reuse the parameters we used for our HTL queries:

In [29]:
from datetime import timedelta, date
from signal_ocean.tonnage_list import VesselClassFilter
from signal_ocean.tonnage_list import PortFilter
from signal_ocean.tonnage_list import (
    VesselFilter,
    PushType,
    MarketDeployment,
    CommercialStatus,
    VesselSubclass,
)

vessel_class_filter = VesselClassFilter(name_like="aframax")
vessel_class = api.get_vessel_classes(vessel_class_filter)[0]

port_filter = PortFilter(name_like="ceyhan")
port = api.get_ports(port_filter)[0]

laycan_end_in_days = 6

vessel_filter = VesselFilter(
    push_types=[PushType.PUSHED],
    market_deployments=[MarketDeployment.RELET, MarketDeployment.SPOT],
    commercial_statuses=[
        CommercialStatus.AVAILABLE,
        CommercialStatus.CANCELLED,
        CommercialStatus.FAILED,
    ],
    vessel_subclass=VesselSubclass.DIRTY,
    latest_ais_since=5,
)


tonnage_list = api.get_tonnage_list(
    port, vessel_class, laycan_end_in_days, vessel_filter
)
tl_data_frame = tonnage_list.to_data_frame()
tl_data_frame

,name,vessel_class,ice_class,year_built,deadweight,length_overall,breadth_extreme,subclass,market_deployment_point_in_time,push_type_point_in_time,...,commercial_operator_point_in_time,commercial_status_point_in_time,eta_point_in_time,latest_ais_point_in_time,open_prediction_accuracy_point_in_time,open_country_point_in_time,open_narrow_area_point_in_time,open_wide_area_point_in_time,availability_port_type_point_in_time,availability_date_type_point_in_time
9457608,Leo Sun,Aframax,NaN,2011,114531,243.80,42,Dirty,Spot,Pushed,...,Zodiac Maritime,Available,2021-11-19 13:00:00+00:00,2021-11-18 06:51:46+00:00,Narrow Area,Egypt,East Mediterranean,Mediterranean,Source,Source
9402263,Seliger,Aframax,NaN,2009,115126,243.80,42,Dirty,Spot,Pushed,...,Navig8 group,Available,2021-11-19 23:00:00+00:00,2021-11-18 15:49:27+00:00,Narrow Area,Egypt,East Mediterranean,Mediterranean,Source,Source
9296195,Paros,Aframax,NaN,2003,107197,247.00,42,Dirty,Spot,Pushed,...,New Shipping,Available,2021-11-21 02:00:00+00:00,2021-11-18 18:08:02+00:00,Narrow Area,Greece,East Mediterranean,Mediterranean,Source,Source
9370836,New Amorgos,Aframax,NaN,2008,107510,243.80,42,Dirty,Spot,Pushed,...,New Shipping,Available,2021-11-21 02:00:00+00:00,2021-11-18 18:06:08+00:00,Narrow Area,Greece,East Mediterranean,Mediterranean,Source,Source
9395331,Corossol,Aframax,NaN,2010,106898,243.80,42,Dirty,Spot,Pushed,...,Cardiff Marine,Available,2021-11-21 07:00:00+00:00,2021-11-18 19:11:04+00:00,Narrow Area,Turkey,Sea of Marmara,Black Sea / Sea Of Marmara,Source,Source
9288710,Delta Captain,Aframax,1A,2005,111009,250.00,44,Dirty,Spot,Pushed,...,Delta Tankers,Available,2021-11-21 08:00:00+00:00,2021-11-18 18:07:12+00:00,Narrow Area,Greece,East Mediterranean,Mediterranean,Source,Source
9348479,Aegean Myth,Aframax,NaN,2006,115838,248.95,44,Dirty,Spot,Pushed,...,Arcadia Shipmanagement,Available,2021-11-21 08:00:00+00:00,2021-11-18 17:35:45+00:00,Port,Greece,East Mediterranean,Mediterranean,Source,Prediction
9439670,Captain A. Stellatos,Aframax,NaN,2010,105849,243.97,42,Dirty,Spot,Pushed,...,Centrofin,Available,2021-11-21 21:00:00+00:00,2021-11-18 16:56:43+00:00,Narrow Area,Italy,Central Mediterranean,Mediterranean,Source,Source
9251822,Willowy,Aframax,NaN,2003,106500,240.50,42,Dirty,Spot,Pushed,...,Hellenic Tankers,Available,2021-11-22 03:00:00+00:00,2021-11-18 15:02:33+00:00,Narrow Area,Egypt,Red Sea,Red Sea,Source,Source
9343340,Gemini Sun,Aframax,NaN,2008,115577,243.80,42,Dirty,Spot,Pushed,...,Zodiac Maritime,Available,2021-11-22 18:00:00+00:00,2021-11-18 18:07:25+00:00,Port,Italy,Central Mediterranean,Mediterranean,Source,Prediction


In [30]:
from signal_ocean.tonnage_list import Column

without_time_zones = (
    tl_data_frame.reset_index()
    .astype(
        {
            Column.OPEN_DATE: "datetime64[ns]",
            Column.ETA: "datetime64[ns]",
            Column.LATEST_AIS: "datetime64[ns]",
        }
    )
)

without_time_zones.to_excel('Ceyhan_Afra_6days_live.xlsx')

C:\Users\PC863~1.ROM\AppData\Local\Temp/ipykernel_20928/255779287.py:4: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  tl_data_frame.reset_index()
